In [ ]:
%matplotlib inline


Group lasso with overlap
========================

Comparison of solvers for a least squares with
overlapping group lasso regularization.




In [ ]:
import numpy as np
from scipy.sparse import linalg as splinalg
from sklearn import datasets
import pylab as plt
import copt as cp

np.random.seed(0)

# .. generate some data ..

n_samples, n_features = 100, 100
blocks_1 = np.arange(n_features) // 8
blocks_2 = np.arange(n_features) // 12


ground_truth = np.zeros(n_features)
ground_truth[blocks_1==4] = 1
ground_truth[blocks_2==3] = 0.5


A = np.random.randn(n_samples, n_features)
p = 0.5
for i in range(1, n_features):
    A[:, i] = p * A[:, i] + (1 - p) * A[:, i-1]
A[:, 0] /= np.sqrt(1 - p ** 2)
sigma = 1.
b = A.dot(ground_truth) + sigma * np.random.randn(n_samples)
# b = np.sign(b)
# b = (b + 1) // 2


# .. compute the step-size ..
max_iter = 5000
s = splinalg.svds(A, k=1, return_singular_vectors=False)[0]
step_size = 1. / cp.utils.get_lipschitz(A, 'square')
f = cp.utils.SquareLoss(A, b)

# .. run the solver for different values ..
# .. of the regularization parameter beta ..
all_betas = [0, 1e-3, 1e-2, 1e-1]
all_trace_ls, all_trace_nols, all_trace_pdhg_nols, all_trace_pdhg = [], [], [], []
all_trace_ls_time, all_trace_nols_time, all_trace_pdhg_nols_time, all_trace_pdhg_time = [], [], [], []
out_img = []
for i, beta in enumerate(all_betas):
    print('beta = %s' % beta)
    G1 = cp.utils.GroupL1(beta, blocks_1)
    G2 = cp.utils.GroupL1(beta, blocks_2)

    def loss(x):
        return f(x) + G1(x) + G2(x)

    cb_tosls = cp.utils.Trace()
    x0 = np.zeros(n_features)
    cb_tosls(x0)
    tos_ls = cp.minimize_TOS(
        f.f_grad, x0, G1.prox, G2.prox, step_size=3 * step_size,
        max_iter=max_iter, tol=1e-14, verbose=1,
        callback=cb_tosls, h_Lipschitz=beta)
    trace_ls = np.array([loss(x) for x in cb_tosls.trace_x])
    all_trace_ls.append(trace_ls)
    all_trace_ls_time.append(cb_tosls.trace_time)

    cb_tos = cp.utils.Trace()
    x0 = np.zeros(n_features)
    cb_tos(x0)
    tos = cp.minimize_TOS(
        f.f_grad, x0, G1.prox, G2.prox,
        step_size=step_size,
        max_iter=max_iter, tol=1e-14, verbose=1,
        backtracking=True, callback=cb_tos)
    trace_nols = np.array([loss(x) for x in cb_tos.trace_x])
    all_trace_nols.append(trace_nols)
    all_trace_nols_time.append(cb_tos.trace_time)
    out_img.append(tos.x)

    cb_pdhg = cp.utils.Trace()
    x0 = np.zeros(n_features)
    cb_pdhg(x0)
    pdhg = cp.gradient.minimize_PDHG(
        f.f_grad, x0, G1.prox, G2.prox,
        callback=cb_pdhg, max_iter=max_iter,
        step_size=step_size,
        step_size2=(1. / step_size) / 2, tol=0, backtracking=False)
    trace_pdhg = np.array([loss(x) for x in cb_pdhg.trace_x])
    all_trace_pdhg.append(trace_pdhg)
    all_trace_pdhg_time.append(cb_pdhg.trace_time)

    cb_pdhg_nols = cp.utils.Trace()
    x0 = np.zeros(n_features)
    cb_pdhg_nols(x0)
    pdhg_nols = cp.gradient.minimize_PDHG(
        f.f_grad, x0, G1.prox, G2.prox,
        callback=cb_pdhg_nols, max_iter=max_iter,
        step_size=step_size,
        step_size2=(1. / step_size) / 2, tol=0, backtracking=False)
    trace_pdhg_nols = np.array([loss(x) for x in cb_pdhg_nols.trace_x])
    all_trace_pdhg_nols.append(trace_pdhg_nols)
    all_trace_pdhg_nols_time.append(cb_pdhg_nols.trace_time)
    #

# .. plot the results ..
fig, ax = plt.subplots(2, 4, sharey=False)
xlim = [4000, 4000, 1000, 100]
markevery = [x//5 for x in xlim]
for i, beta in enumerate(all_betas):
    ax[0, i].set_title(r'$\lambda=%s$' % beta)
    ax[0, i].set_title(r'$\lambda=%s$' % beta)
    ax[0, i].plot(out_img[i])
    ax[0, i].plot(ground_truth)
    ax[0, i].set_xticks(())
    ax[0, i].set_yticks(())

    fmin = min(np.min(all_trace_ls[i]), np.min(all_trace_nols[i]))
    scale = 1. # all_trace_ls[i][0] - fmin
    plot_tos, = ax[1, i].plot(
        (all_trace_ls[i] - fmin) / scale, '--',
        lw=2, marker='o', markevery=markevery[i],
        markersize=10)

    plot_nols, = ax[1, i].plot(
        (all_trace_nols[i] - fmin) / scale,
        lw=2, marker='h', markevery=markevery[i],
        markersize=10)

    plot_pdhg, = ax[1, i].plot(
        (all_trace_pdhg[i] - fmin) / scale,
        lw=2, marker='^', markevery=markevery[i],
        markersize=10)

    plot_pdhg_nols, = ax[1, i].plot(
        (all_trace_pdhg_nols[i] - fmin) / scale,
        lw=2, marker='d', markevery=markevery[i],
        markersize=10)

    ax[1, i].set_xlabel('Iterations')
    ax[1, i].set_yscale('log')
    ax[1, i].set_ylim((1e-10, None))
    ax[1, i].set_xlim((0, xlim[i]))
    ax[1, i].grid(True)


plt.gcf().subplots_adjust(bottom=0.25)
plt.figlegend(
    (plot_tos, plot_nols, plot_pdhg, plot_pdhg_nols),
    ('TOS with line search', 'TOS without line search', 'PDHG LS', 'PDHG no LS'), 'lower center', ncol=2,
    scatterpoints=1, frameon=False,)

ax[1, 0].set_ylabel('Objective minus optimum')
plt.show()